<a href="https://colab.research.google.com/github/udbhav13/2019_DSB/blob/master/2019_DSB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [1]:
from pandas import Series, DataFrame
import pandas as pd
import zipfile
import nbformat
%pylab inline
# !pip install -q <package_name>

Populating the interactive namespace from numpy and matplotlib


In [0]:
train = pd.read_csv('/gdrive/My Drive/data-science-bowl-2019/train.csv')
test = pd.read_csv('/gdrive/My Drive/data-science-bowl-2019/test.csv')
train_labels = pd.read_csv('/gdrive/My Drive/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/gdrive/My Drive/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('/gdrive/My Drive/data-science-bowl-2019/sample_submission.csv')


Filtering training set for installation ids with atleast 1 assessment

In [28]:
# train.columns
# unique_values(train)

assm_event = ['4110','4100']
train_event = train[train['event_code'].isin(assm_event)]
print ('Installation ids with assessment=',len(train_event['installation_id'].drop_duplicates()))
print ('All installation ids=',len (train['installation_id'].drop_duplicates()))

assm_ins_id = train_event['installation_id'].drop_duplicates()
train_flitered = train[train['installation_id'].isin(assm_ins_id)]
train_flitered.shape
# 24% lesser records

Installation ids with assessment= 4248
All installation ids= 17000


(8513498, 11)

Identifying last assessment

In [36]:
train_event.sort_values(['installation_id','game_session','event_code'])
# train_event.set_index('installation_id',inplace=True)
train_event.loc['0006a69f']
# train_flitered.sort_values(['installation_id','game_session','event_code','timestamp'])
# train_flitered

,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world
installation_id,,,,,,,,,,
0006a69f,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
0006a69f,dcb55a27,80d34a30c2998653,2019-08-06T05:24:05.811Z,"{""distance"":8,""round"":0,""event_count"":43,""game...",43,4110,69853,Air Show,Game,TREETOPCITY
0006a69f,dcb55a27,80d34a30c2998653,2019-08-06T05:24:24.726Z,"{""distance"":10,""round"":0,""event_count"":56,""gam...",56,4110,88771,Air Show,Game,TREETOPCITY
0006a69f,14de4c5d,80d34a30c2998653,2019-08-06T05:24:50.323Z,"{""distance"":10,""target_distances"":[5,6,7,8,9,1...",76,4100,114370,Air Show,Game,TREETOPCITY
0006a69f,14de4c5d,80d34a30c2998653,2019-08-06T05:25:11.292Z,"{""distance"":9,""target_distances"":[5,6,7],""corr...",103,4100,135341,Air Show,Game,TREETOPCITY
0006a69f,14de4c5d,80d34a30c2998653,2019-08-06T05:25:37.207Z,"{""distance"":3,""target_distances"":[5,6,7],""corr...",143,4100,161258,Air Show,Game,TREETOPCITY
0006a69f,14de4c5d,80d34a30c2998653,2019-08-06T05:26:01.055Z,"{""distance"":8,""target_distances"":[5,6,7],""corr...",177,4100,185103,Air Show,Game,TREETOPCITY
0006a69f,17113b36,77b8ee947eb84b4e,2019-08-06T05:35:54.898Z,"{""correct"":false,""caterpillars"":[11,8,3],""even...",29,4110,35771,Bird Measurer (Assessment),Assessment,TREETOPCITY
0006a69f,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:01.927Z,"{""correct"":false,""caterpillars"":[11,8,11],""eve...",35,4110,42805,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [31]:
# (train.shape[0] - train_flitered.shape[0])/train.shape[0]
train[train['type']=='Assessment']['event_code'].value_counts()

4030    179134
4070    167454
4020     91181
4025     69117
3010     64931
3110     62858
3021     33526
3121     32976
4100     32743
3020     29708
3120     29039
2020     25752
4040     21989
2000     21239
2030     17424
4035     13501
2010     11691
4110     11158
2025      4759
2035      4040
4090       996
4080       129
Name: event_code, dtype: int64

In [0]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [5]:
train[:10]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK
7,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.546Z,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK
8,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.979Z,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK
9,1325467d,0848ef14a8dc6892,2019-09-06T17:55:08.566Z,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK
